In [81]:
import math
import bisect

from utils import *
import primes

def EaseIoSlope(t, a=2):
    d_base = 1 + 10 ** (a * 0.5)
    return (1.0 / (1 + 10 ** (a * (0.5 - t))) - 1 / (d_base)) / (1.0 - 2 / d_base)

def create_burst(scale, period_cents=1200, theta_scale=2, radius_denominator=0):
    max_edo = 1
    for s in scale:
        if s[1] > max_edo:
            max_edo = s[1]
            continue

    if radius_denominator == 0:
        radius_denominator = max_edo

    rad_scale = math.log2(max_edo)

    theta = [ EaseIoSlope(s[0] / period_cents, theta_scale) * (math.pi * 2) for s in scale ]
    mag = [ math.log2(2 - s[1]/radius_denominator + 1e-4) for s in scale ]

    return (theta, mag)

def et_burst_scale(min_edo=1, max_edo=31, primeFilters=True, primesOnly=True, primeList=[2,3,5,7,11,13,17,19,23,29,31]):
    octave_cents = 1200
    cents = [(0, 1), (octave_cents, 1)]

    primeList.sort()
    max_composite_prime = primeList[-1]

    for edo in range(min_edo, max_edo + 1):
        step = octave_cents / edo

        if primeFilters:
            if primesOnly and not primes.is_prime(edo):
                continue
            
            # skip = True
            # if edo > max_composite_prime:
            #     for primeEdo in primeList:
            #         den = math.gcd(edo, primeEdo)
            #         if den > 1:
            #             skip = False
            #             break
                
            # if skip:
            #     continue

        for n in range(1, edo, 1):
            den = math.gcd(n, edo)
            if den > 1:
                continue
            # cents.append(step * n)
            bisect.insort(cents, (step * n, edo))
    
    return cents

def temp_burst(generator=696.77, period=1200.0, size=7, max_edo=313, mode=0):
    g_2 = generator / period
    cf = get_cf(g_2)
    edo_gens = [(1,1)]
    for i in range(2, len(cf)):
        (gen, edo) = get_convergent(cf, i)
        if edo > max_edo:
            break
        edo_gens.append((gen, edo))

    burst = [ (0, 1), (period, 1) ]
    for (gen,edo) in edo_gens:
        gen_cents = period / edo * gen
        i = mode
        tones = 0
        while tones < size and tones < edo:
             tones += 1
             cents = round((i * gen_cents) % period, 6)
             i += 1
             if cents == 0:
                 continue
             
             bisect.insort(burst, (cents, edo))
        
    return burst


In [ ]:
import matplotlib.pyplot as plot
import matplotlib.projections as projections

import ipywidgets as widgets

min_edo_slider = widgets.IntSlider(1, min=1, max=313)
max_edo_slider = widgets.IntSlider(31*3, min=1, max=313)
generator_slider = widgets.FloatSlider(696.77, min=0.01, max=1200.0, step=0.01)
gen_fine_slider =  widgets.FloatSlider(-20, min=0.01, max=20.0, step=0.01)
size_slider = widgets.IntSlider(7, min=1, max=500)
mode_slider = widgets.IntSlider(0, min=0, max=313)
theta_scale_slider = widgets.FloatSlider(2*0.01, min=0.01, max=10, step=0.01)
radius_scalar_slider = widgets.IntSlider(0, min=0, max=313)

mode_select = widgets.Select(options=["ET", "E-RT"], value="ET")

@widgets.interact(min_edo=min_edo_slider, max_edo=max_edo_slider, theta_a=theta_scale_slider, burst_mode=mode_select, generator=generator_slider,gen_fine=gen_fine_slider, size=size_slider, mode=mode_slider, radius_scalar=radius_scalar_slider)
def draw_burst(**kwArgs):
    
    if kwArgs["burst_mode"] == "E-RT":
        gen = kwArgs["generator"] + kwArgs["gen_fine"]
        scale = temp_burst(generator=gen, size=kwArgs["size"], mode=kwArgs["mode"], max_edo=kwArgs["max_edo"])
    else:
        scale = et_burst_scale(kwArgs["min_edo"], kwArgs["max_edo"])
    
    burst = create_burst(scale, 1200.0, kwArgs["theta_a"], kwArgs["radius_scalar"])

    fig = plot.figure(figsize=(10,10))
    ax = plot.subplot(111, projection='polar')
    ax.set_theta_direction(-1)
    ax.set_theta_offset(math.pi/2)
    ax.set_rlim(0, 1)
    ax.plot(burst[0], burst[1], 'bo-')




interactive(children=(IntSlider(value=1, description='min_edo', max=313, min=1), IntSlider(value=93, descripti…

In [91]:
def get_edo_approx(cents, cents_threshold, max_edo=313, max_depth=100, round_threshold=0.0000001):
    r_2 = round(cents / 1200, 6)
    cf = get_cf(r_2, max_depth, round_threshold)

    edo = 1
    degree = 1
    error = 0

    for i in range(1, len(cf)):
        (degree,edo) = get_convergent(cf, i)

        if edo > max_edo:
            break

        approx = 1200 / edo * degree
        error = cents - approx

        if abs(error) <= cents_threshold:
            break

    return (edo, degree, error)

def make_burst_gradient(resolution=128, period_cents=1200, cents_window=1, max_edo=3130):

    grid = []
    tau = math.pi * 2
    pi_2 = math.pi / 2

    half_res = resolution // 2

    mag_thresh = 1e-8

    overlap = resolution % 2

    for row in range(resolution):
        y = row / half_res
        # ysq = y**2
        y_origin = y - 1
        ysq = y_origin**2

        row = []
        # dbg = []

        # only calculate half due to symmetry
        for col in range(half_res + 1):
            x = col / half_res
            x_origin = x - 1

            mag = math.sqrt(ysq + x_origin**2)

            if mag > 1:
                row.append(0)
                continue

            ang = math.atan2(y_origin, x_origin) + pi_2

            cents = (ang % tau) / tau * period_cents

            (edo, degree, error) = get_edo_approx(cents, cents_window, max_edo)

            edo_scalar = edo / max_edo 

            mag_cf = get_cf(mag, 100)
            mag_error = 0
            mag_edo = 1
            for i in range(1, len(mag_cf)):
                (num, den) = get_convergent(mag_cf, i)
                if den > max_edo:
                    break

                mag_edo = den
                approx = float(num) / mag_edo
                percent = approx / mag
                mag_error = 1 - percent
                if abs(mag_error) <= mag_thresh:
                    break
            
            mag_edo_distance = math.log(1 + abs(mag_edo - edo) / max_edo)

            out = (1.0 - edo_scalar) / (mag_edo_distance + 1)
            # out = (1.0 - edo_scalar)

            row.append(out)
            # dbg.append(cents)


        for col in range(overlap, half_res):
            # row.append(0)
            row.append(row[half_res - col - 1])

        # print(dbg)
        grid.append(row)

    return grid

    

In [ ]:

res_slider=widgets.IntSlider(128, min=8, max=1024)
max_edo_slider=widgets.IntSlider(313, min=1, max=9390)
cents_window_slider=widgets.FloatSlider(1, min=1e-3, max=10, step=1e-6)
@widgets.interact(resolution=res_slider, max_edo=max_edo_slider, cents_window=cents_window_slider)
def draw_burst_gradient(**kwArgs):
    
    grid = make_burst_gradient(resolution=kwArgs["resolution"], max_edo=kwArgs["max_edo"], cents_window=kwArgs["cents_window"])

    plot.figure(figsize=(10,10))
    plot.imshow(grid)

interactive(children=(IntSlider(value=128, description='resolution', max=384, min=-128), IntSlider(value=313, …